In [ ]:
import vip_hci as vip
import os
from hci_plot import plot_frames
import sys
sys.path.append('../../')   # append the path to PyRSM
from PyRSM import PyRSM

In [ ]:
os.chdir('/Users/XXX')

vip.fits.info_fits('./cube_not_normalized.fits')
psfnaco = './naco_betapic_psf.fits'
cube = './cube_not_normalized.fits'
angle = './parang.fits'

angs = vip.fits.open_fits(angle)
cube_orig = vip.fits.open_fits(cube)
psf = vip.fits.open_fits(psfnaco)
pxscale_naco = vip.conf.VLT_NACO['plsc']

In [ ]:
# Measure the FWHM by fitting a 2d Gaussian to the core of the PSF

fit = vip.var.fit_2dgaussian(psf, crop=True, cropsize=9, debug=True)
fwhm = float((fit.fwhm_y+fit.fwhm_x)/2)

# Normalize the PSF flux to one in the FWHM aperture

psfn = vip.metrics.normalize_psf(psf, fwhm, size=19)
psf=  vip.preproc.frame_crop(psfn,11)

In [ ]:
# Center the cube (odd shape)

centy,centx=vip.var.frame_center(cube_orig[0])
cube_recentered, shy1, shx1 = vip.preproc.cube_recenter_2dfit(cube_orig, 
                                                        xy= (int(centx)+1,int(centy)+1),
                                                        fwhm=fwhm, nproc=1, subi_size=5, 
                                                        model='gauss', negative=True, full_output=True,
                                                        debug=False)


In [ ]:
# PyRSM framework (no automated parametrization): Approach 1

# Create PyRSM class object

d=PyRSM(fwhm,minradius=5,maxradius=45,pxscale=pxscale_naco,ncore=1)

# Add a cube

d.add_cube(psf,cube_recentered, angs)

# Add several methods

d.add_method('APCA',interval=[5], intensity='Annulus', distri='A', ncomp=20, var='FR', delta_rot=0.5, asize=5)
d.add_method('NMF',interval=[5], intensity='Annulus', distri='A', ncomp=20, var='FR', asize=5)
d.add_method('LLSG',interval=[5], intensity='Annulus', distri='A', rank=5, var='FR', delta_rot=0.5, asize=5)

# Estimate the cube of likelihoods

d.lik_esti(verbose=True)     

# Estimate final RSM map
     
d.probmap_esti(estimator='Forward',colmode='median')

# Plot final probability map

plot_frames(d.probmap)

In [ ]:
# PyRSM framework (no automated parametrization): Approach 2

# Create PyRSM class object

d=PyRSM(fwhm,minradius=5,maxradius=45,pxscale=pxscale_naco,ncore=1, opti_mode='full-frame')

# Add a cube

d.add_cube(psf,cube_recentered, angs)

# Add several methods

d.add_method('APCA',interval=[5], intensity='Annulus', distri='A', ncomp=20, var='FR', delta_rot=0.5, asize=5)
d.add_method('NMF',interval=[5], intensity='Annulus', distri='A', ncomp=20, var='FR', asize=5)
d.add_method('LLSG',interval=[5], intensity='Annulus', distri='A', rank=5, var='FR', delta_rot=0.5, asize=5)

# Estimate the probability map

d.opti_map(estimator='Forward',colmode='median',threshold=False,Full=True)

# Plot final probability map

plot_frames(d.final_map)

In [ ]:
# auto-RSM framework


# Create PyRSM class object

d=PyRSM(fwhm,minradius=5,maxradius=45,pxscale=pxscale_naco,ncore=1, opti_mode='full-frame',inv_ang=True,opti_type='Contrast',trunc=None)

# Add a cube

d.add_cube(psf,cube_recentered, angs)

# Add several methods

d.add_method('APCA', asize=5,crop_size=3,crop_range=2,opti_bound=[[15,45],[1,4],[0.25,1]],ini_esti=100)
d.add_method('NMF', asize=5,crop_size=3,crop_range=2,opti_bound=[[10,25]])
d.add_method('LLSG', asize=5,crop_size=3,crop_range=2,opti_bound=[[1,10],[1,4]])

# Parameter optimization for the PSF-subtraction techniques

d.opti_model(maxiter=80)

# Parameter optimization for the RSM algorithm

d.opti_RSM(estimator='Forward',colmode='median')

# Optimal selection of the likelihood cubes generated by the RSM algorithm first step
# and estimation of the radial threshold that will be subtracted from the final detection map

d.opti_combination(estimator='Forward',colmode='median',threshold=True,contrast_sel='Max',combination='Bottom-Up') 

# Computation of the final optimal RSM detection map

d.opti_map(estimator='Forward',colmode='median',threshold=True)

# Plot final probability map

plot_frames(d.final_map)

# Save the set of optimal parameters

d.save_parameters('/Users/..','NACO_BetaPic_optimal_param')


In [ ]:
# auto-S/N framework

# Create PyRSM class object

d=PyRSM(fwhm,minradius=5,maxradius=45,pxscale=pxscale_naco,ncore=1, opti_mode='full-frame',inv_ang=True,opti_type='Contrast',trunc=None)

# Add a cube

d.add_cube(psf,cube_recentered, angs)

# Add several methods

d.add_method('APCA', asize=5,opti_bound=[[15,45],[1,4],[0.25,1.5]],ini_esti=60)
d.add_method('NMF', asize=5,opti_bound=[[2,20]])
d.add_method('LLSG', asize=5,opti_bound=[[1,10],[1,4]])

# Parameter optimization for the PSF-subtraction techniques

d.opti_model(maxiter=40)

# Optimal selection of the S/N map generated by the set of selected PSF-subtraction techniques
# and estimation of the radial threshold that will be subtracted from the final S/N map

d.opti_combination(threshold=True,contrast_sel='Max',combination='Bottom-Up',SNR=True) 

# Computation of the final optimal RSM detection map

d.opti_map(threshold=True,SNR=True)

# Plot final probability map

plot_frames(d.final_map)